In [1]:
import sys
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib2
from threading import Thread
from queue import Queue
import math
import requests
import time 
from datetime import timedelta
import pickle
import json

In [2]:
testDataFile = open('testData100k.pckl', 'rb')
testData = pickle.load(testDataFile)

In [3]:
while(True):
    testData.remove(None)

ValueError: list.remove(x): x not in list

In [9]:
testDataTMP = []
features = set()

for i in range(len(testData)):
    testDataTMP.append(testData[i][:17])
    
    for feature in testData[i][17]:
        features.add(feature)

for i in range(len(testData)):
    for feature in features:
        if feature in testData[i][17]:
            testDataTMP[i].append(1)
        else:
            testDataTMP[i].append(0)

In [12]:
columnLabels=["CarID", 
              "Brand",
              "Model",
              "Mileage", 
              "Power", 
              "Fuel", 
              "Transmission", 
              "Registration",
              "Damage",
              "Seats",
              "Doors",
              "Climatisation",
              "Airbags",
              "Color",
              "Interior",
              "ParkAssist",
              "Price"]
columnLabels.extend(features)

['CarID', 'Brand', 'Model', 'Mileage', 'Power', 'Fuel', 'Transmission', 'Registration', 'Damage', 'Seats', 'Doors', 'Climatisation', 'Airbags', 'Color', 'Interior', 'ParkAssist', 'Price', u'Leichtmetallfelgen', u'Elektr. Sitzeinstellung', u'Tagfahrlicht', u'Xenonscheinwerfer', u'Start/Stopp-Automatik', u'Sitzheizung', u'Behindertengerecht', u'Mind. 12 Monate Renault Gebrauchtwagen- und Mobilit\xe4tsgarantie', u'Multifunktionslenkrad', u'Partikelfilter', u'Tempomat', u'Regensensor', u'Navigationssystem', u'Head-Up Display', u'12 Monate Garantie mit Verl\xe4ngerungsm\xf6glichkeit', u'Allradantrieb', u'Panorama-Dach', u'Lederausstattung', u'Sportfahrwerk', u'Garantie', u'Bordcomputer', u'Servolenkung', u'Tuner/Radio', u'Zentralverriegelung', u'Elektr. Fensterheber', u'Skisack', u'Taxi', u'Multi-CD-Wechsler', u'Sportpaket', u'ESP', u'Nichtraucher-Fahrzeug', u'Nebelscheinwerfer', u'ABS', u'24 Monate Garantie: europaweiter Schutz', u'Isofix (Kindersitzbefestigung)', u'Elektr. Seitenspiegel',

In [16]:
testDF = pd.DataFrame(data=testDataTMP, columns=columnLabels)
testDF.head()

,CarID,Brand,Model,Mileage,Power,Fuel,Transmission,Registration,Damage,Seats,...,Traktionskontrolle,Standheizung,Dachreling,MP3-Schnittstelle,Bluetooth,Anhängerkupplung,Freisprecheinrichtung,Schiebedach,Scheckheftgepflegt,Sportsitze
0,233077082,Skoda,Octavia,93.000 km,77 kW (105 PS),Diesel,Automatik,03/2010,NaN,5,...,0,0,1,0,0,0,0,0,1,0
1,235548448,Mercedes-Benz,CLK 200,323.524 km,100 kW (136 PS),Autogas (LPG),Schaltgetriebe,10/1997,Unfallfrei,4,...,0,0,0,1,1,1,1,1,1,1
2,234812981,Mercedes-Benz,C 180,278.000 km,95 kW (129 PS),Benzin,Schaltgetriebe,04/2002,NaN,5,...,0,0,1,0,0,0,0,1,0,0
3,233010839,Opel,Zafira,132.714 km,85 kW (116 PS),Benzin,Schaltgetriebe,07/2009,Unfallfrei,7,...,1,0,1,1,0,1,0,0,1,0
4,228023948,Opel,Meriva,11.583 km,103 kW (140 PS),Benzin,Schaltgetriebe,05/2015,NaN,5,...,1,0,0,1,1,0,1,0,1,0


<h1> Descriptive Analysis </h1>

<h1> Machine Learning </h1>

<h2> Step 1: Data Cleaning/Preparation <h2>
    

In [56]:
dfFinal = pd.DataFrame(columns=["CarID", "Mileage", "Power","Brand", "Model", "Fuel", "Transmission", "Registration", "Price"])
dfFinal = testDF.copy()
#different conversion of Mileage than before
dfFinal['Mileage'] = testDF['Mileage'].str.split('km').str.get(0).str.replace(".", "").astype(int)
dfFinal['Price'] = testDF['Price'].str.split().str.get(0).replace('.', '')
dfFinal['Price'] = dfFinal['Price'].str.replace('.', '').astype(float)
dfFinal['Power'] = testDF['Power'].str.split().str.get(0).astype(float)
dfFinal['Year'] = dfFinal['Registration'].str.extract('(\d\d\d\d)', expand=True)
dfFinal['Year']=pd.to_numeric(dfFinal['Year'])
dfFinal.head()

,CarID,Brand,Model,Mileage,Power,Fuel,Transmission,Registration,Damage,Seats,...,Standheizung,Dachreling,MP3-Schnittstelle,Bluetooth,Anhängerkupplung,Freisprecheinrichtung,Schiebedach,Scheckheftgepflegt,Sportsitze,Year
0,233077082,Skoda,Octavia,93000,77.00,Diesel,Automatik,03/2010,NaN,5,...,0,1,0,0,0,0,0,1,0,2010.00
1,235548448,Mercedes-Benz,CLK 200,323524,100.00,Autogas (LPG),Schaltgetriebe,10/1997,Unfallfrei,4,...,0,0,1,1,1,1,1,1,1,1997.00
2,234812981,Mercedes-Benz,C 180,278000,95.00,Benzin,Schaltgetriebe,04/2002,NaN,5,...,0,1,0,0,0,0,1,0,0,2002.00
3,233010839,Opel,Zafira,132714,85.00,Benzin,Schaltgetriebe,07/2009,Unfallfrei,7,...,0,1,1,0,1,0,0,1,0,2009.00
4,228023948,Opel,Meriva,11583,103.00,Benzin,Schaltgetriebe,05/2015,NaN,5,...,0,0,1,1,0,1,0,1,0,2015.00


### Having look at the correlations

In [57]:
dfFinal.corr()

,Mileage,Power,Price,Leichtmetallfelgen,Elektr. Sitzeinstellung,Tagfahrlicht,Xenonscheinwerfer,Start/Stopp-Automatik,Sitzheizung,Behindertengerecht,...,Standheizung,Dachreling,MP3-Schnittstelle,Bluetooth,Anhängerkupplung,Freisprecheinrichtung,Schiebedach,Scheckheftgepflegt,Sportsitze,Year
Mileage,1.00,0.02,-0.23,-0.15,0.04,-0.40,-0.04,-0.37,-0.21,0.01,...,0.04,-0.04,-0.29,-0.34,0.12,-0.29,0.06,-0.13,-0.02,-0.53
Power,0.02,1.00,0.51,0.32,0.52,0.13,0.50,0.12,0.33,-0.00,...,0.20,0.11,0.11,0.22,0.12,0.26,0.30,0.09,0.34,0.03
Price,-0.23,0.51,1.00,0.15,0.22,0.20,0.24,0.23,0.20,0.00,...,0.11,0.07,0.15,0.23,0.04,0.22,0.12,0.07,0.15,0.14
Leichtmetallfelgen,-0.15,0.32,0.15,1.00,0.26,0.29,0.34,0.26,0.46,-0.02,...,0.09,0.20,0.26,0.35,0.08,0.36,0.16,0.22,0.27,0.27
Elektr. Sitzeinstellung,0.04,0.52,0.22,0.26,1.00,0.15,0.42,0.10,0.30,0.00,...,0.20,0.16,0.13,0.22,0.13,0.26,0.31,0.13,0.20,0.06
Tagfahrlicht,-0.40,0.13,0.20,0.29,0.15,1.00,0.27,0.48,0.39,-0.01,...,0.08,0.21,0.46,0.51,0.01,0.49,0.05,0.25,0.16,0.53
Xenonscheinwerfer,-0.04,0.50,0.24,0.34,0.42,0.27,1.00,0.24,0.41,-0.01,...,0.18,0.19,0.21,0.32,0.13,0.35,0.27,0.18,0.35,0.18
Start/Stopp-Automatik,-0.37,0.12,0.23,0.26,0.10,0.48,0.24,1.00,0.37,-0.01,...,0.06,0.20,0.36,0.45,0.01,0.44,0.04,0.18,0.15,0.46
Sitzheizung,-0.21,0.33,0.20,0.46,0.30,0.39,0.41,0.37,1.00,-0.01,...,0.13,0.25,0.31,0.42,0.11,0.44,0.18,0.25,0.26,0.37
Behindertengerecht,0.01,-0.00,0.00,-0.02,0.00,-0.01,-0.01,-0.01,-0.01,1.00,...,0.03,-0.00,-0.01,-0.01,0.00,-0.01,-0.01,0.01,-0.00,-0.00


### Dropping things we do not need

In [59]:
dfCat = dfFinal.copy()
dfCat = dfCat.drop('CarID', 1)
dfCat = dfCat.drop('Registration', 1)
dfCat = dfCat.drop('Fuel', 1)
#dfCat = dfCat.drop('Decade', 1)

### Repair Function

In [58]:
from sklearn.decomposition import *
def repair_mad(df, n_comp = 4, n_iter = 5, n_remove = None, n_rep = 1):
    num_feat = df.shape[1]
    size = df.shape[0]
    mads = [0.]*num_feat
    cnt = [0]*num_feat
    np.random.seed(191)
    if not n_remove:
        n_rep = 1
    for p in range(n_rep):
        df_prep = df.copy().astype(float)
        if n_remove:
            removed = []
            for i in range(n_remove):
                i = np.random.randint(0,size)
                j = np.random.randint(0,num_feat)
                val = df.iat[i,j]
                df_prep.iat[i,j] = np.nan
                removed.append([i,j,val])
        df_train = df_prep.fillna(df_prep.mean())
        # run PCA and reconstruct data set
        for i in range(n_iter):
            pca = PCA(n_components = n_comp).fit(df_train)
            df_pred = pca.inverse_transform(pca.transform(df_train))
            df_pred = pd.DataFrame(df_pred,columns=df.columns,index=df.index)
            df_train = df_prep.combine_first(df_pred)
        if n_remove:
            for pos in removed:
                diff = pos[2] - df_train.iat[pos[0],pos[1]]
                if not np.isnan(diff):
                    cnt[pos[1]] += 1
                    mads[pos[1]] += 1./cnt[pos[1]]*(abs(diff)-mads[pos[1]])
    if n_remove:
        print pd.DataFrame([mads],index=['MAD'],columns=df_train.columns)
    return df_train

In [60]:
from sklearn.tree import *
from sklearn import cross_validation
from sklearn.tree import *
from sklearn.ensemble import *

### Repairing the data and getting dummies

In [ ]:
#processedData=pd.get_dummies(dfClean)
processedData=pd.get_dummies(dfCat)
#processedData['Year']=yearnummer
repaired=repair_mad(processedData,n_iter=20,n_comp = 10,n_rep = 5)

Xnew = repaired.copy()
Xnew = Xnew.drop('Price', 1)
y = repaired.Price
y=y.values

Xnew['Ones1'] = np.ones(len(repaired))
Xnew['Ones2'] = np.ones(len(repaired))
Xnew=Xnew.values

In [ ]:
reg = RandomForestRegressor(n_estimators=25, oob_score=True)
reg.fit(Xnew,y)
reg.score(Xnew,y),reg.oob_score_